  # Analyzing, Filtering, and Cleaning Aviation Database for the Safest Plane
by Allison Ward, Rick Lataille, and Anthony Mansion

### Project Goal: 
This analysis will idenify those planes with the lowest risk for the U.S. domestic flights market


### Data Source: 
This data is sourced from the National Transportation Safety Board’s (NTSB) Aviation Accident data set from 1962 to 2023. The data contains information regarding civil aviation accidents and selected incidents in the United States and international waters. The raw data set inclues ~90,000 rows of safety-related occurrences involving all types of aircrafts. 

### Limitations:
The dataset was provided by The Flatiron School and contains impurities compared to the dataset available directly from the NTSB. It only shows planes in accidents with no information on flights that were completed without incident.  It does not differentiate between hardware failures and pilot error.

### Import
First import pandas and numpy and import the data into a DataFrame.

In [1]:
#Import the modules we need
import pandas as pd
import numpy as np

# And this is the big data that we will be using
df = pd.read_csv('data/Aviation_Data.csv', low_memory=False)

### Drop unnecessary columns
Many columns in the dataset are unnecessary for this analysis and can be dropped to reduce the size of the file.  Then print the number of items for later reference. 

In [2]:
# Drop the columns we know that we don't need
dropped_columns = ['Event.Id', 'Investigation.Type', 'Accident.Number', 'Airport.Code', 'Airport.Name', \
                   'Aircraft.damage', 'Registration.Number', 'Model', 'FAR.Description', \
                   'Schedule', 'Air.carrier', 'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status', \
                   'Publication.Date']
df.drop(columns = dropped_columns, inplace=True)
print(f"{len(df)} items.")

90348 items.


### Filter columns 
StellarSkies is focused on starting a major airline operating only in the US domestic market for commercial and business flights, so certain types of incidents can be excluded, including:

* Incidents occurring prior to 2013; since then incidents have maintained a consistent level after several decades of declines
* Incidents involving non-airplane aircraft
* Incidents involving amateur-built aircraft
* International flights
* Incidents that occurred while undertaking atypical purposes, such as firefighting, crop-dusting, test flights, etc.
* Incidents that are incorrectly coded as relevant, or with data that is insufficient to include.

In [3]:
# Convert date column to datetime, then filter event dates to include 2013 and later
df['Event.Date'] = pd.to_datetime(df['Event.Date'])
df_filtered = df.loc[df['Event.Date'] >= '2013-01-01']
print(f"Excluding pre-2013 incidents results in {len(df_filtered)} items.")

# Filter aircraft categories for Airplanes only
df_filtered = df_filtered.loc[df_filtered['Aircraft.Category'] == 'Airplane']
print(f"Excluding non-airplane incidents results in {len(df_filtered)} items.")

# Exclude Amateur-built planes
df_filtered = df_filtered.loc[df_filtered['Amateur.Built'] != 'Yes']
print(f"Excluding amateur-built planes results in {len(df_filtered)} items.")

# Exclude certain identified purposes as irrelevant to our stakeholder
allowed_purposes = ['Personal', np.nan, 'Business', 'Executive/corporate', \
                    'Positioning', 'Other Work Use', 'Ferry', 'Unknown', 'Public Aircraft - Federal', \
                   'Public Aircraft - State', 'Public Aircraft - Local', 'Public Aircraft', 'PUBS']
df_filtered = df_filtered.loc[df_filtered['Purpose.of.flight'].isin(allowed_purposes)]
print(f"Excluding irrelevant purposes results in {len(df_filtered)} items.")

# Include only events that happened in the United States or US Territories
allowed_countries = ['United States']
df_filtered = df_filtered.loc[df_filtered['Country'].isin(allowed_countries)]
print(f"Excluding non-US flights results in {len(df_filtered)} items.")

# Filter for foreign locations not noted as foreign using the 'OF' state code in Location
df_filtered['State_Code'] = df_filtered['Location'].str.slice(-2)
df_filtered = df_filtered.loc[df_filtered['State_Code'] != 'OF']
print(f"Excluding foreign flight artifacts results in {len(df_filtered)} items.")

# Drop rows that are missing latitude coordinates (also captures missing Longitude)
df_filtered.dropna(subset=['Latitude'], inplace=True)
print(f"Excluding flights with missing geographic data results in {len(df_filtered)} items.")

Excluding pre-2013 incidents results in 15829 items.
Excluding non-airplane incidents results in 13262 items.
Excluding amateur-built planes results in 11726 items.
Excluding irrelevant purposes results in 9497 items.
Excluding non-US flights results in 7320 items.
Excluding foreign flight artifacts results in 7307 items.
Excluding flights with missing geographic data results in 7302 items.


### Data transformation
Several new columns will simplify the analysis for the safety, large jets and small jets analyses.

In [4]:
# Use dt functions to extract year, month and day of week and create new columns
df_filtered['Year'] = df['Event.Date'].dt.year
df_filtered['Month'] = df['Event.Date'].dt.month
df_filtered['Day_Of_Week'] = df['Event.Date'].dt.day_name()

# Create a new column to simplify the large jet analysis
separate_large_jets = ["Airbus", "Boeing", "Embraer"]
df_filtered['Large_Jets'] = df_filtered['Make'].map(lambda x: "Other" if x not in separate_large_jets else x)

# Create a new column to simplify the large jet analysis
separate_small_jets = ["Bombardier", "Dassault", "Gulfstream", "Honda", "Textron"]
df_filtered['Small_Jets'] = df_filtered['Make'].map(lambda x: "Other" if x not in separate_small_jets else x)

# Create a new column summing fatal and serious injuries
df_filtered['Major_Injuries'] = df_filtered['Total.Fatal.Injuries'] + df_filtered['Total.Serious.Injuries']

Once those filters are applied, several more columns can be dropped since they are no longer relevant.

In [5]:
# Drop columns that are no longer useful
obsolete_columns = ['Event.Date', 'Location', 'Country', 'Aircraft.Category', 'Amateur.Built']
df_filtered.drop(columns = obsolete_columns, inplace=True)

### Consolidating Makes
Entries in the make column reflect manual data entries that must be cleaned in order to accurately reflect the current manufacturer.  The consolidation is limited to the major participants in the commercial and business markets.

In [6]:
# Record original makes for later comparison
Original_makes = len(df_filtered['Make'].unique())

In [7]:
# These map functions will clean the 'Make' column to focus on the makes in our analysis
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Airbus" if x.lower().strip()[:6]=="airbus" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Airbus" if x.lower().strip()[:5]=="fouga" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Boeing" if x.lower().strip()[:6]=="boeing" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Boeing" if x.lower().strip()[:9]=="mcdonnell" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Boeing" if x.lower().strip()[:7]=="douglas" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Boeing" if x.lower().strip()[:8]=="rockwell" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Bombardier" if x.lower().strip()[:10]=="bombardier" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Bombardier" if x.lower().strip()[:5]=="gates" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Bombardier" if x.lower().strip()[:7]=="learjet" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Bombardier" if x.lower().strip()[:8]=="canadair" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Dassault" if x.lower().strip()[:8]=="dassault" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Embraer" if x.lower().strip()[:7]=="embraer" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Gulfstream" if x.lower().strip()[:10]=="gulfstream" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Gulfstream" if x.lower().strip()[:3]=="iai" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Honda" if x.lower().strip()[:5]=="honda" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Textron" if x.lower().strip()[:6]=="cessna" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Textron" if x.lower().strip()[:4]=="rath" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Textron" if x.lower().strip()[:4]=="rayt" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Textron" if x.lower().strip()[:7]=="textron" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Textron" if x.lower().strip()[:5]=="beech" else x)
df_filtered['Make'] = df_filtered['Make'].map(lambda x: "Textron" if x.lower().strip()[:6]=="hawker" else x)

In [8]:
# Show the amount of consolidation in makes
print(f"The original {Original_makes} makes have been reduced to {len(df_filtered['Make'].unique())} makes.")

The original 670 makes have been reduced to 595 makes.


### Imputing unknown values

In [9]:
# Change "NaN" to 'None' or 'Unknown', as appropriate
df_filtered['Injury.Severity'].fillna('None', inplace=True)
df_filtered['Purpose.of.flight'].fillna('Unknown', inplace=True)
df_filtered['Engine.Type'].fillna('Unknown', inplace=True)
df_filtered['Number.of.Engines'].fillna('Unknown', inplace=True)

Convert entries in the 'Make' column to Title case for readability in the visuals

In [10]:
# Put all Makes into Title case, for readability
df_filtered['Make'] = df_filtered['Make'].map(lambda x: x.title())

### Finalize Filtered Data
Save the now-filtered data to a new CSV file which:
* preserves the information in the raw data file for later re-use or confirmation
* creates a new, smaller data file for use in Tableau

In [11]:
# Write to a new CSV file/overwrites CSV file
df_filtered.to_csv('data/Filtered_Aviation_Data.csv', index=False)

## Conclusion
This analysis shows:
1. Reciprocating engines and single-engine planes present the biggest risk of major injury and should be avoided.
2. Boeing, Cessna and Bombardier have worse safety records than Airbus, Embraer, Gulfstream and Dassault.
3. Lastly, summers and weekends are more dangerous and protocols should be set in place keeping this in mind.